# Instalando os pacotes faltantes

In [0]:
!apt install libspatialindex-dev
!pip install osmnx

# Baixando os dados

[Porto Taxi Service Trajectory](https://archive.ics.uci.edu/ml/datasets/Taxi+Service+Trajectory+-+Prediction+Challenge,+ECML+PKDD+2015)

In [0]:
!wget -O porto.zip 'https://archive.ics.uci.edu/ml/machine-learning-databases/00339/train.csv.zip'

In [0]:
!unzip porto.zip

# Importando bibliotecas necessárias

* [Pandas](https://pandas.pydata.org/)
* [GeoPandas](https://geopandas.org/)
* [OSMNX](https://osmnx.readthedocs.io/en/stable/index.html)
* [tqdm](https://tqdm.github.io/)

In [0]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from tqdm import tqdm

# Processamento dos dados

In [0]:
df = pd.read_csv('train.csv', nrows=1_000)

In [0]:
df.head()

In [0]:
df['POLYLINE'][0]

In [0]:
df[df['POLYLINE'] == '[]']

In [0]:
df = df[df['POLYLINE'] != '[]']

In [0]:
def get_trajectories_points(df):
    lat = []
    lon = []
    for p in df['POLYLINE'].values:
        coords = p[2:-2].split('],[')
        for c in coords:
            point = c.split(',')
            lon.append(float(point[0]))
            lat.append(float(point[1]))
    return pd.DataFrame({'x': lon, 'y': lat})

In [0]:
data = get_trajectories_points(df)

In [0]:
data

# Grafo da rede de tráfego

In [0]:
def get_graph_from_place(place, nw, w):
    G = ox.graph_from_place(place, network_type=nw, which_result=w)
    N, E = ox.graph_to_gdfs(G, nodes=True, edges=True)
    return G, N, E

In [0]:
G, N, E = get_graph_from_place('Porto, Portugal', 'drive', 2)

In [0]:
ox.plot_graph(G)

In [0]:
N.head()

In [0]:
E.head()

In [0]:
E['index'] = E.index
E['density'] = 1
E.drop(
    labels=['lanes', 'ref', 'maxspeed', 'access', 'width', 
            'tunnel', 'bridge', 'junction', 'area', 'service'], 
    axis=1, inplace=True
)

In [0]:
E.head()

In [0]:
N.shape, E.shape

# [Map matching](https://www.wikiwand.com/en/Map_matching)

In [0]:
def get_nearest_edges_from_dataframe(df):
    nearest_edges = ox.geo_utils.get_nearest_edges(
        G, 
        df['x'].values, df['y'].values, 
        method='balltree'
    )
    return nearest_edges

In [0]:
nearest_edges = get_nearest_edges_from_dataframe(data)

In [0]:
nearest_edges, len(nearest_edges)

In [0]:
ox.plot_graph_routes(G, nearest_edges, route_linewidth=2, route_color='blue',
                     orig_dest_node_size=25, orig_dest_node_color='green')

In [0]:
def get_new_edges(nearest_edges):
    edges = []
    for u, v in tqdm(nearest_edges):
        edges.append(E[(E['u'] == u) & (E['v'] == v)].sort_values(by='length').values[0])
    new_E = gpd.GeoDataFrame(edges, columns=E.columns)
    return new_E

In [0]:
new_E = get_new_edges(nearest_edges)

In [0]:
new_E.shape

# Dense paths

In [0]:
new_E_dropped = new_E.drop_duplicates(['index']).copy()

In [0]:
new_E_dropped.shape

In [0]:
new_E_dense = new_E.groupby('index').sum()

In [0]:
new_E_dense

In [0]:
new_E_dropped.head()

In [0]:
new_E_dropped['density'] = new_E_dropped['index'].apply(lambda x: new_E_dense.loc[x]['density'])

In [0]:
new_E_dropped.head()

In [0]:
new_E_dropped.shape, new_E_dropped[new_E_dropped['density'] >= 50].shape

In [0]:
def get_dense_paths(new_E, thresh):
    new_E_dropped = new_E.drop_duplicates(['index']).copy()
    new_E_dense = new_E.groupby('index').sum()
    new_E_dropped['density'] = new_E_dropped['index'].apply(lambda x: new_E_dense.loc[x]['density'])
    new_E_dropped = new_E_dropped[new_E_dropped['density'] >= thresh]
    return new_E_dropped

In [0]:
new_E_dense = get_dense_paths(new_E, 50)

In [0]:
def save_new_graph(new_E):
    new_G = ox.gdfs_to_graph(N, new_E)
    return new_G

In [0]:
dense_G = save_new_graph(new_E_dense)

In [0]:
ox.plot_graph(dense_G, edge_color='red', node_color='lightgray')

# Mapa interativo

In [0]:
routes_G = save_new_graph(new_E)

In [0]:
m = ox.plot_graph_folium(routes_G, edge_color='gray', edge_width=1)

In [0]:
dm = ox.plot_graph_folium(dense_G, m, popup_attribute='density',
                          edge_width=5, edge_color='red')

In [0]:
dm.save('porto_taxi_dense_paths_map.html')